In [1]:
import pandas as pd
import numpy as np
import pulp
import copy
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

In [2]:
def opt_port(df_sample,epsilon):
    epsilon = epsilon
    X_tr = df_sample.copy(deep=True)

    mto_van_opt = df_sample[df_sample.flg_max_vane==1]['mto_e'].sum()
    mto_opt_port = mto_van_opt*epsilon

    idcs = list(df_sample['codclavecic'].unique())
    df_sample.set_index(keys=['codclavecic','escenario'], inplace=True)

    esc_opt = pulp.LpVariable.dicts("esc_opt",
                                    ((idc, escenario) for idc, escenario in df_sample.index),
                                    cat='Binary')

    # Definición del Modelo
    model_opt = pulp.LpProblem("Optimización_del_Van_Esperado", pulp.LpMaximize)

    # Función Objetivo
    model_opt += pulp.lpSum([esc_opt[idc, escenario] * df_sample.loc[(idc, escenario), 'van_e'] for idc, escenario in df_sample.index]) 

    # Restricciones
    for idc in idcs:
        escenarios = X_tr['escenario'][X_tr['codclavecic']==idc]
        model_opt += pulp.lpSum(esc_opt[(idc, esc)] for esc in escenarios) == 1

    model_opt += pulp.lpSum([esc_opt[idc, escenario] * df_sample.loc[(idc, escenario), 'mto_e'] for idc, escenario in df_sample.index]) >= mto_opt_port

    model_opt.solve()


    print(pulp.LpStatus[model_opt.status])
    print(model_opt.objective.value())


    output = []
    for idc, esc in esc_opt:
        var_output = {
            'idc': idc, 
            'escenario': esc, 
            'binario': esc_opt[(idc, esc)].varValue, 
            'value':df_sample.loc[(idc,esc),'van_e'] }
        
        output.append(var_output) 

    output_df = pd.DataFrame.from_records(output)
    return output_df

In [3]:
route_bases = Path('.').resolve() / 'data' / 'bases'
route_output = Path('.').resolve() / 'data' / 'optimizacion'

input_filename = "wtp_escenarios_ind_20231229.csv"
output_filename_5 = f"salida_opt_port_campania_{(datetime.now() + relativedelta(months=1)).strftime('%Y%m')}_5_new.csv"
output_filename_10 = f"salida_opt_port_campania_{(datetime.now() + relativedelta(months=1)).strftime('%Y%m')}_10_new.csv"


output_filename_5_final, extension = output_filename_5.rsplit('.', 1)  
output_filename_5_final = output_filename_5_final.replace("new", "escenario_new")  
output_filename_5_final = f"{output_filename_5_final}.{extension}" 

output_filename_10_final, extension = output_filename_10.rsplit('.', 1)  
output_filename_10_final = output_filename_10_final.replace("new", "escenario_new")  
output_filename_10_final = f"{output_filename_10_final}.{extension}" 


full_filename_input = route_bases / input_filename
full_filename_output_5 = route_output/ output_filename_5
full_filename_output_10 = route_output/ output_filename_10
full_filename_output_5_final = route_output/ output_filename_5_final
full_filename_output_10_final = route_output/ output_filename_10_final

In [4]:
# Load DataFrame
df = pd.read_csv(full_filename_input)

# Lista de Clientes
clientes = set(df['codclavecic'])

# Número de samples que deseas generar
samples_num = 10

# Tamaño de las submuestras
sample_size = int(len(clientes)/samples_num)

# Lista de Dataframes
df_list = []
output_list = []
tiempo = []

porc = 1.05

print("Clientes totales del DataFrame:",len(df.codclavecic.unique()),"\n")

# generar las submuestras sin repetir elementos
for i in range(1,samples_num+1):
    if i<samples_num: 
        clientes_sample = random.sample(clientes, sample_size)
        df_sample = df[df.codclavecic.isin(clientes_sample)]
        df_sample["sample_num"] = i 
        df = df.drop(df_sample.index)

        print("Clientes del DataFrame del grupo",i,":",len(df_sample.codclavecic.unique()))
        print("Clientes que quedan en el DataFrame:", len(df.codclavecic.unique()))
       
    
        df_list.append(df_sample)
        clientes = clientes - set(clientes_sample)

        start_time = datetime.now() 

        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)
        
        time_elapsed = datetime.now() - start_time  
        tiempo.append(time_elapsed)
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
    else:
        df_sample = df.copy()
        df_sample["sample_num"] = i    
        df_list.append(df_sample)
        
        print("Clientes del último sample:",len(df_sample.codclavecic.unique()))
        
        start_time = datetime.now() 
       
        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        tiempo.append(time_elapsed)
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
      
df_tot = pd.concat(df_list)   
df_tot_opt = pd.concat(output_list)

Clientes totales del DataFrame: 18579 

Clientes del DataFrame del grupo 1 : 1857
Clientes que quedan en el DataFrame: 16722
Optimal
155837.59817939054
Time elapsed (hh:mm:ss.ms) 0:00:30.833632, grupo:1 

Clientes del DataFrame del grupo 2 : 1857
Clientes que quedan en el DataFrame: 14865
Optimal
158707.93770185372
Time elapsed (hh:mm:ss.ms) 0:00:26.436268, grupo:2 

Clientes del DataFrame del grupo 3 : 1857
Clientes que quedan en el DataFrame: 13008
Optimal
153460.84167399217
Time elapsed (hh:mm:ss.ms) 0:00:25.346703, grupo:3 

Clientes del DataFrame del grupo 4 : 1857
Clientes que quedan en el DataFrame: 11151
Optimal
155705.05576288415
Time elapsed (hh:mm:ss.ms) 0:00:15.027070, grupo:4 

Clientes del DataFrame del grupo 5 : 1857
Clientes que quedan en el DataFrame: 9294
Optimal
155774.96984727803
Time elapsed (hh:mm:ss.ms) 0:00:26.084832, grupo:5 

Clientes del DataFrame del grupo 6 : 1857
Clientes que quedan en el DataFrame: 7437
Optimal
162534.73409855523
Time elapsed (hh:mm:ss.ms

In [5]:
df_tot.shape

(259177, 9)

In [6]:
df_tot_opt.to_csv(full_filename_output_5,index=False)

In [7]:
df_tot_opt = pd.read_csv(full_filename_output_5)

#Salida del WTP Individual Escenarios
df_wtp_esc = pd.read_csv(full_filename_input)

#Salida de la Optimización del Portafolio tomando los leads por grupo
df_opt_port_group = df_tot_opt[df_tot_opt.binario==1][['idc','escenario']].rename(columns={'idc':'codclavecic'})

#Se le pega la TEA Opt por grupo de acuerdo al escenario
df_tea_opt_port_by_group = pd.merge(df_opt_port_group, df_wtp_esc[['codclavecic','escenario','tea', 'cluster','tir']], on=['codclavecic','escenario'], how='left')

df_tea_opt_port_by_group.to_csv(full_filename_output_5_final, index=False)

In [8]:
df_tea_opt_port_by_group.shape

(18579, 5)

In [14]:
# Load DataFrame
df = pd.read_csv(full_filename_input)

# Lista de Clientes
clientes = set(df['codclavecic'])

# Número de samples que deseas generar
samples_num = 10

# Tamaño de las submuestras
sample_size = int(len(clientes)/samples_num)

# Lista de Dataframes
df_list = []
output_list = []

porc = 1.061

print("Clientes totales del DataFrame:",len(df.codclavecic.unique()),"\n")

# generar las submuestras sin repetir elementos
for i in range(1,samples_num+1):
    if i<samples_num:
        clientes_sample = random.sample(clientes, sample_size)
        df_sample = df[df.codclavecic.isin(clientes_sample)]
        df_sample["sample_num"] = i 
        df = df.drop(df_sample.index)

        print("Clientes del DataFrame del grupo",i,":",len(df_sample.codclavecic.unique()))
        print("Clientes que quedan en el DataFrame:", len(df.codclavecic.unique()))
       
    
        df_list.append(df_sample)
        clientes = clientes - set(clientes_sample)

        start_time = datetime.now() 

        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
    else:
        df_sample = df.copy()
        df_sample["sample_num"] = i    
        df_list.append(df_sample)
        
        print("Clientes del último sample:",len(df_sample.codclavecic.unique()))
        
        start_time = datetime.now() 
       
        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
      
df_tot_10 = pd.concat(df_list)   
df_tot_opt_10 = pd.concat(output_list) 

Clientes totales del DataFrame: 18579 

Clientes del DataFrame del grupo 1 : 1857
Clientes que quedan en el DataFrame: 16722
Optimal
137064.396979098
Time elapsed (hh:mm:ss.ms) 0:00:27.943870, grupo:1 

Clientes del DataFrame del grupo 2 : 1857
Clientes que quedan en el DataFrame: 14865
Optimal
133502.8731850735
Time elapsed (hh:mm:ss.ms) 0:00:28.597663, grupo:2 

Clientes del DataFrame del grupo 3 : 1857
Clientes que quedan en el DataFrame: 13008
Optimal
146382.3465535692
Time elapsed (hh:mm:ss.ms) 0:00:14.047463, grupo:3 

Clientes del DataFrame del grupo 4 : 1857
Clientes que quedan en el DataFrame: 11151
Optimal
143269.92845842056
Time elapsed (hh:mm:ss.ms) 0:00:19.788370, grupo:4 

Clientes del DataFrame del grupo 5 : 1857
Clientes que quedan en el DataFrame: 9294
Optimal
128798.93850335418
Time elapsed (hh:mm:ss.ms) 0:00:09.821486, grupo:5 

Clientes del DataFrame del grupo 6 : 1857
Clientes que quedan en el DataFrame: 7437
Optimal
146767.73621654997
Time elapsed (hh:mm:ss.ms) 0:

In [15]:
df_tot_opt_10.to_csv(full_filename_output_10,index=False)

In [16]:
df_tot_opt_10.shape

(259177, 4)

In [17]:
df_tot_opt_10 = pd.read_csv(full_filename_output_10)

#Salida del WTP Individual Escenarios
df_wtp_esc = pd.read_csv(full_filename_input)

#Salida de la Optimización del Portafolio tomando los leads por grupo
df_opt_port_group = df_tot_opt_10[df_tot_opt_10.binario==1][['idc','escenario']].rename(columns={'idc':'codclavecic'})

#Se le pega la TEA Opt por grupo de acuerdo al escenario
df_tea_opt_port_by_group = pd.merge(df_opt_port_group, df_wtp_esc[['codclavecic','escenario','tea', 'cluster','tir']], on=['codclavecic','escenario'], how='left')

df_tea_opt_port_by_group.to_csv(full_filename_output_10_final, index=False)

In [18]:
df_tea_opt_port_by_group.shape

(18579, 5)